In [ ]:
from binance.client import Client
import pandas as pd
import numpy as np
from hmmlearn.hmm import GaussianHMM
import matplotlib.pyplot as plt

# Binance API credentials
API_KEY = 'your_api_key'
API_SECRET = 'your_api_secret'

# Initialize Binance client
client = Client(API_KEY, API_SECRET)

# Fetch historical data
def fetch_historical_data(symbol, interval, start_str):
    klines = client.get_historical_klines(symbol, interval, start_str)
    data = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    data.set_index('timestamp', inplace=True)
    data = data[['open', 'high', 'low', 'close', 'volume']].astype(float)
    return data

symbol = 'BTCUSDT'
interval = Client.KLINE_INTERVAL_1HOUR
start_str = '1 Jan 2021'
data = fetch_historical_data(symbol, interval, start_str)

In [ ]:
# Prepare data for HMM
data['returns'] = data['close'].pct_change()
data['log_returns'] = np.log1p(data['returns'])
data.dropna(inplace=True)

# Train HMM
hmm = GaussianHMM(n_components=3, covariance_type="full", n_iter=1000)
hmm.fit(data[['log_returns']])

# Predict hidden states
hidden_states = hmm.predict(data[['log_returns']])
data['hidden_state'] = hidden_states

In [ ]:
# Generate trading signals based on hidden states
data['signal'] = 0
data.loc[data['hidden_state'] == 0, 'signal'] = 1  # Buy signal
data.loc[data['hidden_state'] == 2, 'signal'] = -1  # Sell signal

In [ ]:
# Backtest the strategy
initial_balance = 10000
balance = initial_balance
position = 0
data['portfolio_value'] = balance

for i in range(1, len(data)):
    if data['signal'].iloc[i] == 1 and position == 0:
        position = balance / data['close'].iloc[i]
        balance = 0
    elif data['signal'].iloc[i] == -1 and position > 0:
        balance = position * data['close'].iloc[i]
        position = 0
    data['portfolio_value'].iloc[i] = balance + position * data['close'].iloc[i]

# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(data['close'], label='BTC-USD Price')
plt.plot(data['portfolio_value'], label='Portfolio Value')
plt.legend()
plt.show()

# Print final portfolio value
print(f"Final portfolio value: ${data['portfolio_value'].iloc[-1]:.2f}")